In [94]:
import tensorflow as tf

In [95]:
tf.__version__

'2.10.1'

## 1. 데이터 로드

- os를 이용한 데이터 로드

In [96]:
import numpy as np
import pandas as pd
import os
import random
import re
from bs4 import BeautifulSoup

train_data_path = './aclImdb/train/'
test_data_path = './aclImdb/test/'

positive_dir = 'pos/'
negative_dir = 'neg/'

def refine(text):
    lower_text = text.lower()
    soup_text = BeautifulSoup(lower_text).get_text()
    refine_text =  re.sub('[^A-Za-z0-9가-힣 ]+', '', soup_text)
    return refine_text

def dir_to_dataset(filepath, inputs, labels, label):

    for filename in os.listdir(filepath):
        if filename.endswith('.txt'):
            with open(os.path.join(filepath, filename),'r', encoding='utf-8') as f:
                data = f.read()
                str_data = refine(data)
                inputs.append(str_data)
                labels.append(label)

train_text_data = []
train_label_data = []

dir_to_dataset(train_data_path + positive_dir, train_text_data, train_label_data, 1)
dir_to_dataset(train_data_path + negative_dir, train_text_data, train_label_data, 0)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_9488\3691449413.py:16: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup_text = BeautifulSoup(lower_text).get_text()


In [97]:
print(len(train_text_data), len(train_label_data))

25000 25000


In [98]:
test_text_data = []
test_label_data = []

dir_to_dataset(test_data_path + positive_dir, test_text_data, test_label_data, 1)
dir_to_dataset(test_data_path + negative_dir, test_text_data, test_label_data, 0)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_9488\3691449413.py:16: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup_text = BeautifulSoup(lower_text).get_text()


In [99]:
print(len(test_text_data), len(test_label_data))

25000 25000


In [100]:
comb_data_train = list(zip(train_text_data, train_label_data))
random.shuffle(comb_data_train)
train_text_data, train_label_data = zip(*comb_data_train)

comb_data_test = list(zip(test_text_data, test_label_data))
random.shuffle(comb_data_test)
test_text_data, test_label_data = zip(*comb_data_test)

## 2. 텍스트 데이터 전처리
 - (1) 텍스트 정제
 
    - 소문자 변환

    - html 태그 제거
    
    - 구두점 제거

- (2) 토큰화 
 - vocab 파일을 word_index  변환 후 그것으로 토큰화 진행

In [101]:
from tensorflow.keras.preprocessing.text import Tokenizer

vocab = './aclImdb/imdb.vocab'

# vocab 파일 열기
with open(vocab, 'r') as f:
    word = [line.strip() for line in f]
# 단어 딕셔너리 생성
word_index = {word : i for i, word in enumerate(word)}

# 단어 딕셔너리 이용한 토큰화
def index_tokenize(text):
    words = text.split()
    return [word_index.get(word, 0) for word in words]

# 시퀀스 배열
train_sequences = []
# 트레인 셋 토큰화
for data in train_text_data:
    train_sequences.append(index_tokenize(data))

test_sequences = []
for data in test_text_data:
    test_sequences.append(index_tokenize(data))

In [102]:
for i in range(10):
    print(test_text_data[i])
    print(test_sequences[i])
    print(test_label_data[i])

queen of the damned is one of the best vampire movies i had ever seen the movie had suspense action and gore the combination of the fierce demanding attitude of the queen and the rock mood of our star very well acted by stuart townsend makes a wonderfully done movie that only this combination can create im always the one to give advice to my friends and family members on which movies are worthy of renting and when they ask me if queen of the damned is worthy i tell them its worthy of buying this movie is most for sure a musthave in all horror movie lovers homes
[1545, 3, 0, 6204, 5, 26, 3, 0, 115, 1327, 99, 8, 65, 121, 107, 0, 15, 65, 812, 206, 1, 593, 0, 2134, 3, 0, 8454, 6048, 2110, 3, 0, 1545, 1, 0, 719, 1265, 3, 253, 324, 52, 71, 991, 30, 5166, 9536, 161, 2, 1616, 222, 15, 10, 60, 9, 2134, 66, 956, 4502, 204, 0, 26, 4, 192, 1973, 4, 54, 350, 1, 209, 1028, 18, 59, 99, 22, 1655, 3, 2689, 1, 50, 33, 896, 68, 42, 1545, 3, 0, 6204, 5, 1655, 8, 358, 92, 89, 1655, 3, 2604, 9, 15, 5, 87, 1

### sequence 분석

 - 시퀀스 길이를 시각화하여 분석한 결과 시퀀스 길이가 2500에 근접한 시퀀스배열이 극 소수 있음 (1개) - 242번 인덱스의 시퀀스 배열
    - 1500 ~ 2000 은 최소 5개의 시퀀스가 존재
    - 제거는 추후 테스트 셋을 고려하지 못한 상태에 이를 수 있음
##### 결론 : 2450의 시퀀스 길이가 최대이기에 위의 값을 통해 시퀀스 패딩을 진행한다. 

In [103]:
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.sequence import pad_sequences

sequences_len = []
# 시퀀스 길이 배열
for i in range(len(train_sequences)):
    sequences_len.append(len(train_sequences[i]))
    if sequences_len[i] > 2000:
        print(sequences_len[i])

max_seq_len = 2450

padding_train_sequence = pad_sequences(train_sequences, padding='post')
padding_test_sequence = pad_sequences(test_sequences, maxlen=2450, padding='post')

2450


- numpy 배열 형태로 변환
- 2차원은 일반적으로 lstm에서 차원을 지정하고 인풋값으로 넣어줘야함
    - lstm 은 일반적으로 3차원 입력을 기대함

In [104]:
train_label_data = np.array(train_label_data)
test_label_data = np.array(test_label_data)

padding_train_sequence = np.array(padding_train_sequence)
padding_test_sequence = np.array(padding_test_sequence)

In [105]:
vocab_size = len(word_index)

#### 모델 구현

In [128]:
from tensorflow.keras.layers import LSTM, Dropout, BatchNormalization, Dense, Embedding, GlobalAveragePooling1D

class base_model(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dims, first_dim, second_dim):
        
        super(base_model, self).__init__()

        self.embedding = Embedding(vocab_size, embedding_dims)

        self.pooling_layer = GlobalAveragePooling1D()

        self.first_dense = Dense(first_dim, activation='relu')

        self.final_dense = Dense(second_dim, activation='sigmoid')

    def call(self, inputs):

        inputs = tf.cast(inputs, dtype=tf.float32)

        x = self.embedding(inputs)

        x = self.pooling_layer(x)

        x = self.first_dense(x)

        return self.final_dense(x)
    

In [129]:
base_model = base_model(vocab_size, 16, 24, 1)
base_model.build(input_shape=(None, 2450))
base_model.summary()

Model: "base_model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_15 (Embedding)    multiple                  1432432   
                                                                 
 global_average_pooling1d_8   multiple                 0         


 (GlobalAveragePooling1D)                                        
                                                                 
 dense_23 (Dense)            multiple                  408       
                                                                 
 dense_24 (Dense)            multiple                  25        
                                                                 
Total params: 1,432,865
Trainable params: 1,432,865
Non-trainable params: 0
_________________________________________________________________


In [130]:
base_model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')

In [131]:
embedding_record = base_model.fit(x = padding_train_sequence,
                                  y = train_label_data,
                                  validation_data=(padding_test_sequence, test_label_data),
                                  epochs=50)

Epoch 1/50
782/782 [==============================] - 2s 3ms/step - loss: 0.6894 - accuracy: 0.5474 - val_loss: 0.6727 - val_accuracy: 0.5778
Epoch 2/50
782/782 [==============================] - 2s 2ms/step - loss: 0.5640 - accuracy: 0.7670 - val_loss: 0.4781 - val_accuracy: 0.8080
Epoch 3/50
782/782 [==============================] - 2s 2ms/step - loss: 0.3882 - accuracy: 0.8559 - val_loss: 0.3694 - val_accuracy: 0.8602
Epoch 4/50
782/782 [==============================] - 2s 2ms/step - loss: 0.3059 - accuracy: 0.8855 - val_loss: 0.3376 - val_accuracy: 0.8648
Epoch 5/50
782/782 [==============================] - 2s 2ms/step - loss: 0.2651 - accuracy: 0.8998 - val_loss: 0.3141 - val_accuracy: 0.8740
Epoch 6/50
782/782 [==============================] - 2s 2ms/step - loss: 0.2334 - accuracy: 0.9134 - val_loss: 0.3029 - val_accuracy: 0.8782
Epoch 7/50
782/782 [==============================] - 2s 2ms/step - loss: 0.2096 - accuracy: 0.9228 - val_loss: 0.2928 - val_accuracy: 0.8820
Epoch 

In [ ]:
from tensorflow.keras.layers import LSTM, Dropout, BatchNormalization, Dense, Embedding, GlobalAveragePooling1D

class lstm_model(tf.keras.Model):
    def __init__(self, word_idx, embedding_dims, first_unit):
        super(lstm_model, self).__init__()

        self.embedding_layer = Embedding(word_idx, embedding_dims)

        self.first_lstm = LSTM(first_unit, dropout=0.3)

        self.dense = Dense(1, activation='sigmoid')

    def call(self, inputs):

        inputs = tf.cast(inputs, dtype=tf.float32)

        x = self.embedding_layer(inputs)

        x = self.first_lstm(x)

        return self.dense(x)


In [ ]:
from tensorflow.keras.optimizers import Adam

first_units = 128
vocab_size = len(word_index)
embedding_dims = 16

adam = Adam(learning_rate=0.01)

model = lstm_model(word_idx=vocab_size, embedding_dims=embedding_dims, first_unit=first_units)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
model.build(input_shape=(None, 2450))
model.summary()

Model: "lstm_model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    multiple                  1432432   
                                                                 
 lstm_6 (LSTM)               multiple                  74240     
                                                                 
 dense_14 (Dense)            multiple                  129       
                                                                 
Total params: 1,506,801
Trainable params: 1,506,801
Non-trainable params: 0
_________________________________________________________________


In [ ]:
one_lstm_record = model.fit(x = padding_train_sequence,
                            y= train_label_data, 
                            validation_data=(padding_test_sequence, test_label_data), 
                            epochs=50)

Epoch 1/50
782/782 [==============================] - 105s 133ms/step - loss: 0.6933 - accuracy: 0.5015 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/50
782/782 [==============================] - 100s 128ms/step - loss: 0.6932 - accuracy: 0.5021 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 3/50
782/782 [==============================] - 100s 129ms/step - loss: 0.6932 - accuracy: 0.5007 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 4/50
782/782 [==============================] - 103s 132ms/step - loss: 0.6932 - accuracy: 0.4951 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 5/50
782/782 [==============================] - 102s 130ms/step - loss: 0.6932 - accuracy: 0.4989 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 6/50
782/782 [==============================] - 115s 147ms/step - loss: 0.6932 - accuracy: 0.4976 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 7/50
782/782 [==============================] - 133s 170ms/step - loss: 0.6932 - accuracy: 0.4970 - val_loss: 0.6931 -

KeyboardInterrupt: 

In [ ]:
def draw_graph(history, name):
    fig, ax1 = plt.subplots()

    plt.title(name + ' acc evaluate')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('accuracy')
    ax1.plot(history.history['accuracy'], color='blue')
    ax1.plot(history.history['val_accuracy'],color='red')
    ax1.tick_params(axis='y',)
    ax1.legend(['acc', 'val_acc'], loc='upper left')

    ax2 = ax1.twinx()  # x축을 공유하는 두 번째 축을 생성합니다.

    ax2.set_ylabel('loss')
    ax2.plot(history.history['loss'], color='blue', linestyle='dashed')
    ax2.plot(history.history['val_loss'], color='red', linestyle='dashed')
    ax2.tick_params(axis='y')
    ax2.legend(['loss', 'val_loss'], loc='upper right')

    fig.tight_layout()  # 그래프가 겹치지 않도록 레이아웃을 조정합니다.
    plt.show()

2450


In [ ]:
draw_graph(one_lstm_record, 'unity-lstm')